In [1]:
# %%
import pyspark
print("Spark version:", pyspark.__version__)

Spark version: 3.5.4


In [2]:
# %%
import sys
print("Python version:", sys.version)

Python version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


In [3]:
# %%
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("CheckHadoopVersion").getOrCreate()

# Obtener la versión de Hadoop
hadoop_version = spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion()
print("Hadoop version:", hadoop_version)

# Detener la sesión de Spark
spark.stop()

Hadoop version: 3.3.4


In [4]:
# %%
from pyspark.sql import SparkSession

def validate_spark_functionality():
    # Crear una sesión de Spark
    spark = SparkSession.builder.appName("SimpleSparkValidation").getOrCreate()
    
    # Crear un DataFrame simple
    data = [("Alice", 34), ("Bob", 45), ("Cathy", 29)]
    columns = ["Name", "Age"]
    df = spark.createDataFrame(data, columns)
    
    # Mostrar el DataFrame
    df.show()
    
    # Realizar una operación simple: calcular la edad promedio
    avg_age = df.groupBy().avg("Age").collect()[0][0]
    print("Average Age:", avg_age)
    
    # Detener la sesión de Spark
    spark.stop()

# Llamar a la función para validar la funcionalidad de Spark
validate_spark_functionality()

+-----+---+
| Name|Age|
+-----+---+
|Alice| 34|
|  Bob| 45|
|Cathy| 29|
+-----+---+

Average Age: 36.0


In [1]:
# %%
import os
os.chdir('..')  # Cambia al directorio raíz del proyecto

# Ejecuta el módulo pdf_processor
!python -m src.data.pdf_processor

In [6]:
# 3. Ejecuta la herramienta de etiquetado
# %%
import os
import sys

# Imprimir la ruta actual
print("Ruta actual:", os.getcwd())

# Cambia al directorio raíz del proyecto
os.chdir('c:/Users/Steev/Documents/AiranFranco/tributai  IA/document_classifier')

# Imprimir la nueva ruta actual
print("Nueva ruta actual:", os.getcwd())

# Agrega el directorio src al PYTHONPATH
sys.path.append(os.path.abspath('src'))

# Imprimir el PYTHONPATH
print("PYTHONPATH:", sys.path)

# Ejecuta el módulo labeling_tool
!python -m src.data.labeling_tool

Ruta actual: c:\Users\Steev\Documents
Nueva ruta actual: c:\Users\Steev\Documents\AiranFranco\tributai  IA\document_classifier
PYTHONPATH: ['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\python310.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib', 'C:\\Users\\Steev\\AppData\\Local\\Microsoft\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0', 'c:\\Users\\Steev\\Documents\\AiranFranco\\tributai  IA\\document_classifier\\.venv', '', 'c:\\Users\\Steev\\Documents\\AiranFranco\\tributai  IA\\document_classifier\\.venv\\lib\\site-packages', 'c:\\Users\\Steev\\Documents\\AiranFranco\\tributai  IA\\document_classifier\\.venv\\lib\\site-packages\\win32', 'c:\\Users\\Steev\\Documents\\AiranFranco\\tributai  IA\\document_classifier\\.venv\\lib\\site-packages\

INFO:__main__:Página 0 marcada como primera página
INFO:__main__:Página 10 marcada como primera página
INFO:__main__:Página 20 marcada como primera página
INFO:__main__:Etiquetas guardadas correctamente
INFO:__main__:Todos los PDFs han sido etiquetados
INFO:__main__:Etiquetas guardadas correctamente


In [7]:
import json
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Cargar etiquetas
with open("data/labels/labels.json", "r") as f:
    labels = json.load(f)

# Preparar datos
X = []  # imágenes
y = []  # etiquetas

for pdf_name, first_pages in labels.items():
    pdf_dir = f"data/processed/{pdf_name}"
    for page_num in range(len(os.listdir(pdf_dir))):
        # Cargar imagen
        img_path = f"{pdf_dir}/page_{page_num}.png"
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (224, 224))
        img = img / 255.0  # normalizar
        
        # Agregar etiqueta
        label = 1 if page_num in first_pages else 0
        
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)

# Dividir en entrenamiento y validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

FileNotFoundError: [Errno 2] No such file or directory: 'data/labels/labels.json'

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

# Crear modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entrenar
history = model.fit(X_train, y_train, 
                   epochs=10, 
                   validation_data=(X_test, y_test))

# Guardar modelo
model.save('data/models/page_classifier.h5')

In [ ]:
def predict_first_pages(pdf_path):
    # Convertir PDF a imágenes
    pages = convert_from_path(pdf_path)
    
    predictions = []
    for i, page in enumerate(pages):
        # Preprocesar imagen
        img = np.array(page)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (224, 224))
        img = img / 255.0
        img = np.expand_dims(img, axis=[0, -1])
        
        # Predecir
        pred = model.predict(img)[0][0]
        if pred > 0.5:
            predictions.append(i)
    
    return predictions